In [65]:
from statsbombpy import sb
import pandas as pd
import numpy as np
import networkx as nx

In [66]:
df = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/MathildeStouby/opt/anaconda3/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarni

In [67]:
xT = pd.read_csv("https://raw.githubusercontent.com/AKapich/WorldCup_App/main/app/xT_Grid.csv", header=None)
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

In [68]:
def get_xT(df, event_type):
    df = df[df['type']==event_type]
    
    df['x'], df['y'] = zip(*df['location'])
    df['end_x'], df['end_y'] = zip(*df[f'{event_type.lower()}_end_location'])

    df[f'start_x_bin'] = pd.cut(df['x'], bins=xT_cols, labels=False)
    df[f'start_y_bin'] = pd.cut(df['y'], bins=xT_rows, labels=False)
    df[f'end_x_bin'] = pd.cut(df['end_x'], bins=xT_cols, labels=False)
    df[f'end_y_bin'] = pd.cut(df['end_x'], bins=xT_rows, labels=False)
    df['start_zone_value'] = df[[f'start_x_bin', f'start_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
    df['end_zone_value'] = df[[f'end_x_bin', f'end_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
    df['xT'] = df['end_zone_value']-df['start_zone_value']

    return df[['xT', 'minute', 'period', 'team', 'match_id']]

In [69]:
from copy import deepcopy
aux_df  = deepcopy(df)

aux_df = aux_df[aux_df['type']=='Pass']

In [70]:
aux_df['x'], aux_df['y'] = zip(*aux_df['location'])
aux_df['end_x'], aux_df['end_y'] = zip(*aux_df[f'pass_end_location'])

aux_df[f'start_x_bin'] = pd.cut(aux_df['x'], bins=xT_cols, labels=False)
aux_df[f'start_y_bin'] = pd.cut(aux_df['y'], bins=xT_rows, labels=False)
aux_df[f'end_x_bin'] = pd.cut(aux_df['end_x'], bins=xT_cols, labels=False)
aux_df[f'end_y_bin'] = pd.cut(aux_df['end_x'], bins=xT_rows, labels=False)

In [71]:
aux_df['start_zone_value'] = aux_df[[f'start_x_bin', f'start_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
aux_df['end_zone_value'] = aux_df[[f'end_x_bin', f'end_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)

aux_df['xT'] = aux_df['end_zone_value']-aux_df['start_zone_value']

/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_86413/1174840208.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux_df['start_zone_value'] = aux_df[[f'start_x_bin', f'start_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_86413/1174840208.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  aux_df['end_zone_value'] = aux_df[[f'end_x_bin', f'end_y_bin']].apply(lambda z: xT[z[1]][z[0]], axis=1)


In [72]:
xT_data = pd.concat([get_xT(df=df, event_type='Pass'), get_xT(df=df, event_type='Carry')], axis=0)
xT_data = xT_data[xT_data['xT'] > 0.001]
xT_data = xT_data[xT_data['minute'] <= 90]
xT_data = xT_data [~((xT_data['period'] == 1) & (xT_data['minute'] > 44 ))]

/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_86413/3939404028.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['x'], df['y'] = zip(*df['location'])
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_86413/3939404028.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['x'], df['y'] = zip(*df['location'])
/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_86413/3939404028.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

In [73]:
xT_data

,xT,minute,period,team,match_id
207,0.002097,0,1,Werder Bremen,3895302
208,0.005633,0,1,Werder Bremen,3895302
209,0.001001,0,1,Werder Bremen,3895302
210,0.003050,0,1,Werder Bremen,3895302
212,0.007266,0,1,Werder Bremen,3895302
...,...,...,...,...,...
109951,0.001901,89,2,RB Leipzig,3895052
109953,0.005819,89,2,RB Leipzig,3895052
109956,0.002154,90,2,Bayer Leverkusen,3895052
109959,0.002032,90,2,RB Leipzig,3895052


In [74]:
#Bin minutes into time intervals so minute 3 is now minute 0, minute 14 is minute 10 and so on
xT_data['minute_interval'] = ((xT_data['minute']) // 5) * 5 
xT_data = xT_data[(xT_data['minute_interval']<90) & (xT_data['minute_interval']>0) & (xT_data['minute_interval']!= 45)]
#Group by the team, match and minute interval and calculate the sum of xT for each
xT_sums = xT_data.groupby(["team","match_id",'minute_interval'])['xT'].sum().reset_index()

In [75]:
xT_data

,xT,minute,period,team,match_id,minute_interval
295,0.006060,5,1,Werder Bremen,3895302,5
296,0.003075,5,1,Bayer Leverkusen,3895302,5
297,0.004152,5,1,Bayer Leverkusen,3895302,5
298,0.001368,5,1,Werder Bremen,3895302,5
301,0.003853,5,1,Werder Bremen,3895302,5
...,...,...,...,...,...,...
109945,0.002604,88,2,RB Leipzig,3895052,85
109947,0.001368,89,2,RB Leipzig,3895052,85
109948,0.003050,89,2,RB Leipzig,3895052,85
109951,0.001901,89,2,RB Leipzig,3895052,85


In [76]:
#Filter for the xT values of Bayer Leverkusen
bayer_xt = xT_sums[xT_sums['team'] == 'Bayer Leverkusen']
#Make new column name before merging 
bayer_xt = bayer_xt.rename(columns={'xT': 'bayer_xT'})

df_merged = xT_sums.merge(bayer_xt, on=['match_id', 'minute_interval'], how='left')

#Calculate momentum for each row, also in the rows where Bayer Lverkusen's "xT" and "bayer_xt" is the same (explain later)
df_merged['momentum'] = df_merged['bayer_xT'] - df_merged['xT']

#Remove rows where momentum is 0 as this is where xT of Bayer Leverkusen is subtracted from itself 
#(There is not an instance where Bayer Leverkusen xT and opponent xT is equal so this isnt a problem)
momentum_df = df_merged[df_merged["momentum"]!=0]

In [77]:
#Make ids based off of the game and minute interval
momentum_df["id"] = momentum_df["match_id"].astype(str) + momentum_df["minute_interval"].astype(str)

/var/folders/_3/q0p4rhwn0r31w83n5ds1htx40000gn/T/ipykernel_86413/3464649215.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_df["id"] = momentum_df["match_id"].astype(str) + momentum_df["minute_interval"].astype(str)


In [78]:
momentum_df = momentum_df[["id","momentum"]]

In [79]:
momentum_df

,id,momentum
0,38951945,0.051752
1,389519410,0.086223
2,389519415,0.171258
3,389519420,-0.096416
4,389519425,0.151314
...,...,...
1082,389526665,0.076905
1083,389526670,0.062513
1084,389526675,0.161248
1085,389526680,0.168021


In [80]:
momentum_df.to_csv("momentum_data.csv", index=False)